<a href="https://colab.research.google.com/github/SamSzamocki/SamSzamocki_playground/blob/main/HF_Tut_Ch2_3_finetune_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
!pip install h5py
!pip install typing-extensions
!pip install wheel
!pip install datasets evaluate transformers[sentencepiece]

!pip uninstall -y numpy pandas pyarrow

!pip install numpy==1.23.5 pandas==2.0.3 pyarrow==15.0.0
# Uninstall the packages to avoid potential conflicts
!pip uninstall transformers accelerate -y

# Reinstall the required packages
!pip install transformers[torch]
!pip install accelerate -U

# Import and initialize TrainingArguments
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")
print("TrainingArguments initialized successfully")
"""

'\n!pip install h5py\n!pip install typing-extensions\n!pip install wheel\n!pip install datasets evaluate transformers[sentencepiece]\n\n!pip uninstall -y numpy pandas pyarrow\n\n!pip install numpy==1.23.5 pandas==2.0.3 pyarrow==15.0.0\n# Uninstall the packages to avoid potential conflicts\n!pip uninstall transformers accelerate -y\n\n# Reinstall the required packages\n!pip install transformers[torch]\n!pip install accelerate -U\n\n# Import and initialize TrainingArguments\nfrom transformers import TrainingArguments\n\ntraining_args = TrainingArguments("test-trainer")\nprint("TrainingArguments initialized successfully")\n'

In [3]:
!pip install h5py typing-extensions wheel
!pip install datasets evaluate transformers[sentencepiece]
!pip uninstall -y numpy pandas pyarrow
!pip install numpy==1.23.5 pandas==2.0.3 pyarrow==15.0.0
!pip uninstall transformers accelerate -y
!pip install transformers[torch]
!pip install accelerate -U

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
Found existing installation: pandas 2.0.3
Uninstalling pandas-2.0.3:
  Successfully uninstalled pandas-2.0.3
Found existing installation: pyarrow 15.0.0
Uninstalling pyarrow-15.0.0:
  Successfully uninstalled pyarrow-15.0.0
  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached pandas-2.0.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Using cached pyarrow-15.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (38.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.0 which is incompatible.
google-colab 1.0.0 requires req

Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: accelerate 0.31.0
Uninstalling accelerate-0.31.0:
  Successfully uninstalled accelerate-0.31.0
  Using cached transformers-4.41.2-py3-none-any.whl (9.1 MB)
  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)


In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")
print("TrainingArguments initialized successfully")


TrainingArguments initialized successfully


In [5]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, Trainer

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
